In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
import calendar
import datetime
import pickle

In [2]:
#reads csv
df = pd.read_csv("invoices_tester.csv")
df.drop(columns=['itemName', 'itemDesc'], errors='ignore', inplace=True)
df

,invoiceID,customerID,itemID,itemPrice,itemQTY,totalCost,dateInvoiced
0,1,1,111121,34.99,35,1224.65,2022-01-01
1,2,2,111148,9.99,83,829.17,2022-01-01
2,3,3,111155,9.99,30,299.70,2022-01-02
3,4,4,111145,7.99,40,319.60,2022-01-02
4,5,5,111122,14.99,94,1409.06,2022-01-02
...,...,...,...,...,...,...,...
4470,4471,4471,111135,12.99,98,1273.02,2024-03-29
4471,4472,4472,111121,34.99,75,2624.25,2024-03-29
4472,4473,4473,111137,3.99,68,271.32,2024-03-29
4473,4474,4474,111132,59.99,47,2819.53,2024-03-29


In [3]:
#sets date invoiced as index then removes column
df.index = pd.to_datetime(df['dateInvoiced'])
df.drop(columns=['dateInvoiced'], inplace=True)
df.index = pd.DatetimeIndex(df.index).to_period('D')
df

,invoiceID,customerID,itemID,itemPrice,itemQTY,totalCost
dateInvoiced,,,,,,
2022-01-01,1,1,111121,34.99,35,1224.65
2022-01-01,2,2,111148,9.99,83,829.17
2022-01-02,3,3,111155,9.99,30,299.70
2022-01-02,4,4,111145,7.99,40,319.60
2022-01-02,5,5,111122,14.99,94,1409.06
...,...,...,...,...,...,...
2024-03-29,4471,4471,111135,12.99,98,1273.02
2024-03-29,4472,4472,111121,34.99,75,2624.25
2024-03-29,4473,4473,111137,3.99,68,271.32


In [4]:
#gets all item IDs
items = df['itemID'].unique()
print(items)

[111121 111148 111155 111145 111122 111146 111113 111135 111160 111137
 111141 111115 111142 111111 111157 111152 111134 111158 111126 111131
 111144 111130 111129 111150 111114 111151 111119 111156 111138 111127
 111123 111140 111120 111132 111143 111112 111153 111154 111125 111147
 111116 111159 111139 111136 111133 111117 111124 111149 111118 111128]


In [5]:
dfList=[]

#splits df into array of dataframes based off item ID
for ID in items:
    temp = df[df['itemID'] == ID].copy()
    dfList.append(temp)

In [6]:
splitY=[]

#extracts the item QTY from each df in array
for item in dfList:
    splitY.append(item[['itemQTY']])

In [7]:
dateRange = []
extendedSplit = []

#gets range of dates for each item df
for split in splitY:
    dateRange.append(pd.date_range(start=split.index[0].to_timestamp(), end=split.index[-1].to_timestamp()))
    
#converts ranges to dataframe
for ranges in dateRange:
    extended_split_data = pd.DataFrame(index=pd.concat([pd.Series(ranges)]))
    extendedSplit.append(extended_split_data)

#adds corrosponding item QTY or 0 if no matching found
for i, split in enumerate(extendedSplit):
    for index in split.index:
        # Check if the index exists in the itemQTY dataframe
        if index in splitY[i].index:
            split.loc[index, 'itemQTY'] = splitY[i].loc[index].values[0]
        else:
            split.loc[index, 'itemQTY'] = 0

   

print(extendedSplit[0])

            itemQTY
2022-01-01     35.0
2022-01-02      0.0
2022-01-03      0.0
2022-01-04      0.0
2022-01-05      0.0
...             ...
2024-03-25      0.0
2024-03-26     66.0
2024-03-27      0.0
2024-03-28      0.0
2024-03-29     75.0

[819 rows x 1 columns]


In [8]:
#enable for testing

df_dropped_last_30 = []

for split in extendedSplit:
    df_dropped_last_30.append(split.iloc[-30:])
    split.drop(split.tail(30).index, inplace=True)

In [9]:
models = []
fitted = []
i = 0

#creates model and adds fitted to array
for split in extendedSplit:
    split.index = pd.DatetimeIndex(split.index).to_period('D')
    model = SARIMAX(split, order=(10, 0, 0), seasonal_order=(1, 0, 1, 12), trend='ct')
    models.append(model)
    fitted.append(model.fit(maxiter=200))
    print("model", i, "trained")
    i += 1
print(fitted[0])

model 0 trained
model 1 trained
model 2 trained
model 3 trained
model 4 trained
model 5 trained
model 6 trained


C:\Users\Nicolas\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
C:\Users\Nicolas\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


model 7 trained
model 8 trained
model 9 trained
model 10 trained
model 11 trained
model 12 trained
model 13 trained
model 14 trained
model 15 trained
model 16 trained
model 17 trained
model 18 trained
model 19 trained
model 20 trained
model 21 trained
model 22 trained
model 23 trained
model 24 trained
model 25 trained
model 26 trained
model 27 trained
model 28 trained
model 29 trained
model 30 trained
model 31 trained
model 32 trained
model 33 trained
model 34 trained
model 35 trained
model 36 trained
model 37 trained
model 38 trained
model 39 trained
model 40 trained
model 41 trained
model 42 trained
model 43 trained
model 44 trained
model 45 trained
model 46 trained
model 47 trained
model 48 trained
model 49 trained


In [10]:
forecast = []

#getting the qty forecast from fitted models
for fit in fitted:
    forecast.append(fit.forecast(steps=30))

In [11]:
holderFore = []
i=0

#combine the days into prediction for 1 month
for fore in forecast:
    holderPred = 0
    for pred in fore:
        holderPred += pred
    holderFore.append(round(holderPred))
    i += 1
    
print(holderFore)

[154, 238, 206, 247, 128, 151, 189, 238, 136, 115, 196, 160, 206, 213, 103, 203, 242, 142, 124, 163, 163, 311, 211, 152, 173, 194, 162, 195, 114, 200, 208, 198, 212, 242, 173, 190, 176, 118, 162, 125, 238, 216, 197, 192, 190, 201, 118, 254, 226, 237]


In [12]:
#enable for testing

holderReal = []
i=0
for real in df_dropped_last_30:
    holderReal.append(0)
    for qty in real['itemQTY']:
        holderReal[i] += qty
    i += 1
    
print(holderReal)


[141.0, 257.0, 298.0, 81.0, 414.0, 130.0, 428.0, 458.0, 159.0, 166.0, 114.0, 175.0, 177.0, 324.0, 114.0, 133.0, 170.0, 103.0, 109.0, 303.0, 112.0, 136.0, 355.0, 485.0, 42.0, 79.0, 182.0, 294.0, 225.0, 176.0, 358.0, 241.0, 63.0, 346.0, 290.0, 233.0, 231.0, 237.0, 463.0, 142.0, 62.0, 230.0, 290.0, 245.0, 169.0, 214.0, 96.0, 267.0, 265.0, 232.0]


In [13]:
#enable for testing

l=0
while l < len(items):
    mae = mean_absolute_error(df_dropped_last_30[l], forecast[l])
    print(f"Mean Absolute Error: {mae}")
    l += 1

Mean Absolute Error: 9.186726732912456
Mean Absolute Error: 14.375436241646167
Mean Absolute Error: 15.322389415949226
Mean Absolute Error: 9.866182492849013
Mean Absolute Error: 16.630405383268037
Mean Absolute Error: 8.718724948379414
Mean Absolute Error: 17.208441250575717
Mean Absolute Error: 20.158077374438154
Mean Absolute Error: 8.923411071949788
Mean Absolute Error: 8.84548629393776
Mean Absolute Error: 9.431374548696168
Mean Absolute Error: 10.06431943914944
Mean Absolute Error: 11.810840667469234
Mean Absolute Error: 15.512641263664026
Mean Absolute Error: 6.904762002739356
Mean Absolute Error: 10.29512417342131
Mean Absolute Error: 12.631470272209272
Mean Absolute Error: 7.53161649529629
Mean Absolute Error: 6.905078264861655
Mean Absolute Error: 14.003306266814873
Mean Absolute Error: 8.07004585226615
Mean Absolute Error: 12.473395655815414
Mean Absolute Error: 15.475435956646368
Mean Absolute Error: 18.960094226339177
Mean Absolute Error: 6.841238562599449
Mean Absolute Er

In [14]:
collected = []

#combines item ID and combined predictions into an array
for item, holderPred in zip(items, holderFore):
    collected.append([item, holderPred])
    
print(collected)

[[111121, 154], [111148, 238], [111155, 206], [111145, 247], [111122, 128], [111146, 151], [111113, 189], [111135, 238], [111160, 136], [111137, 115], [111141, 196], [111115, 160], [111142, 206], [111111, 213], [111157, 103], [111152, 203], [111134, 242], [111158, 142], [111126, 124], [111131, 163], [111144, 163], [111130, 311], [111129, 211], [111150, 152], [111114, 173], [111151, 194], [111119, 162], [111156, 195], [111138, 114], [111127, 200], [111123, 208], [111140, 198], [111120, 212], [111132, 242], [111143, 173], [111112, 190], [111153, 176], [111154, 118], [111125, 162], [111147, 125], [111116, 238], [111159, 216], [111139, 197], [111136, 192], [111133, 190], [111117, 201], [111124, 118], [111149, 254], [111118, 226], [111128, 237]]
